In [ ]:
import pandas as pd

df = pd.read_csv("D:/PSU/DS 440/DataSet/ini_train_5.csv")

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.isnull().values.any()

In [ ]:
df.max().max()

In [ ]:
import numpy as np
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df =df.dropna()

In [ ]:
df.info()

In [ ]:
dummies = pd.get_dummies(df[['Blur','Info','Occlusion','Collage','Action','Near']])

In [ ]:
dummies.info()

In [ ]:
print(dummies.head())

In [ ]:
y = df['Pawpularity']

In [ ]:
X_numerical = df.drop(['train_id', 'Pawpularity','Blur','Info','Occlusion','Collage','Action','Near'], axis=1).astype('float16')

In [ ]:
list_numerical = X_numerical.columns
list_numerical

In [ ]:
X = pd.concat([X_numerical, dummies[['Blur','Info','Occlusion','Collage','Action','Near']]], axis=1)
X.info()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

In [ ]:
X_train.head()

In [ ]:
from sklearn.linear_model import Lasso

reg = Lasso(alpha=1)
reg.fit(X_train, y_train)

In [ ]:
print('R squared training set', round(reg.score(X_train, y_train)*100, 2))
print('R squared test set', round(reg.score(X_test, y_test)*100, 2))

In [ ]:
from sklearn.metrics import mean_squared_error

# Training data
pred_train = reg.predict(X_train)
mse_train = mean_squared_error(y_train, pred_train)
print('MSE training set', round(mse_train, 2))

# Test data
pred = reg.predict(X_test)
mse_test =mean_squared_error(y_test, pred)
print('MSE test set', round(mse_test, 2))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

alphas = np.linspace(0.01,10,100)
lasso = Lasso(max_iter=10000)
coefs = []

for a in alphas:
    lasso.set_params(alpha=a)
    lasso.fit(X_train, y_train)
    coefs.append(lasso.coef_)

ax = plt.gca()

ax.plot(alphas, coefs)
ax.set_xscale('log')
plt.axvline(
   0.0586877794238647, linestyle="--", color="k", label="alpha: CV estimate"
)
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('Standardized Coefficients')
plt.title('Lasso coefficients as a function of alpha');

In [ ]:
from sklearn.linear_model import LassoCV

# Lasso with 10 fold cross-validation
model = LassoCV(cv=10, random_state=0, max_iter=10000)

# Fit model
model.fit(X_train, y_train)

In [ ]:
model.alpha_

In [ ]:
# Set best alpha
lasso_best = Lasso(alpha=model.alpha_)
lasso_best.fit(X_train, y_train)

In [ ]:
print(list(zip(lasso_best.coef_, X)))

In [ ]:
print('R squared training set', round(lasso_best.score(X_train, y_train)*100, 2))
print('R squared test set', round(lasso_best.score(X_test, y_test)*100, 2))

In [ ]:
mean_squared_error(y_test, lasso_best.predict(X_test))

In [ ]:
import matplotlib.pyplot as plt
y = y_test
x = lasso_best.predict(X_test)
plt.scatter(x, y, s = 5)
plt.xlabel('Lasso Output')
plt.ylabel('Pawpularity')

plt.semilogx(model.alphas_, model.mse_path_, ":")
plt.plot(
    model.alphas_ ,
    model.mse_path_.mean(axis=-1),
    "k",
    label="Average across the folds",
    linewidth=2,
)
plt.axvline(
    model.alpha_, linestyle="--", color="k", label="alpha: CV estimate"
)

plt.legend()
plt.xlabel("alphas")
plt.ylabel("Mean square error")
plt.title("Mean square error on each fold")
plt.axis("tight")

ymin, ymax = 300, 475
plt.ylim(ymin, ymax);

In [ ]:
plt.semilogx(model.alphas_, model.mse_path_, ":")
plt.plot(
    model.alphas_ ,
    model.mse_path_.mean(axis=-1),
    "k",
    label="Average across the folds",
    linewidth=2,
)
plt.axvline(
    model.alpha_, linestyle="--", color="k", label="alpha: CV estimate"
)

plt.legend()
plt.xlabel("alphas")
plt.ylabel("Mean square error")
plt.title("Mean square error on each fold")
plt.axis("tight")

ymin, ymax = 300, 475
plt.ylim(ymin, ymax);